In [1]:
# Import the dependencies.
from citipy import citipy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime
import requests

# Import the API key.
from config import weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
#coordinates

In [4]:
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

749

In [5]:
#Perform an API call with the OpenWeatherMap.
url = "https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [8]:
#Retrieve the following information from the API call:

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        #time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_descr = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        #city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_descr})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | hermanus
Processing Record 2 of Set 1 | port moresby
Processing Record 3 of Set 1 | provideniya
Processing Record 4 of Set 1 | batagay-alyta
Processing Record 5 of Set 1 | pevek
Processing Record 6 of Set 1 | bafq
Processing Record 7 of Set 1 | vardo
Processing Record 8 of Set 1 | hobart
Processing Record 9 of Set 1 | hobyo
Processing Record 10 of Set 1 | ushuaia
Processing Record 11 of Set 1 | pathein
Processing Record 12 of Set 1 | kodiak
Processing Record 13 of Set 1 | longyearbyen
Processing Record 14 of Set 1 | palabuhanratu
City not found. Skipping...
Processing Record 15 of Set 1 | price
Processing Record 16 of Set 1 | baykit
Processing Record 17 of Set 1 | kapaa
Processing Record 18 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 19 of Set 1 | huazolotitlan
City not found. Skipping...
Processing Record 20 of Set 1 | lagoa
Processing Record 21 of Set 1 | lebu
Processin

In [9]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hermanus,ZA,-34.4187,19.2345,59.81,82,99,9.62,overcast clouds
1,Port Moresby,PG,-9.4431,147.1797,85.42,66,75,5.75,broken clouds
2,Provideniya,RU,64.3833,-173.3000,28.90,51,15,12.21,few clouds
3,Batagay-Alyta,RU,67.8006,130.4114,19.89,92,100,12.77,light snow
4,Pevek,RU,69.7008,170.3133,21.15,71,100,8.08,overcast clouds
5,Bafq,IR,31.6128,55.4107,67.57,50,1,2.93,clear sky
6,Vardo,NO,70.3705,31.1107,28.17,73,100,8.70,overcast clouds
7,Hobart,AU,-42.8794,147.3294,68.25,57,75,19.57,broken clouds
8,Hobyo,SO,5.3505,48.5268,83.79,71,100,16.49,light rain
9,Ushuaia,AR,-54.8000,-68.3000,38.86,87,20,9.22,few clouds


In [10]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")